In [ ]:
# 用實際的 ( x , y ) 配合 Gradient Descend 演算法來求解迴歸，不用模組


import numpy


# 1. 先產生 20 組隨機 ( x , y ) 資料，並假設真實模型為 y = 2x + 1

# a. x 隨機產生 0 到 10 中間 20 個點

x = numpy.linspace(0,10,20)

# b. 再利用 y = 2x + 1 + noise 來產生隨機 y , Y 因為是 X 產生，所以也是 numpy array

# numpy.random.random(n) 為 產生 n 個 0 ~ 1 之間的隨機數字的產生器

y = 2 * x + 1 + numpy.random.random(20)


# 2. 定義目標 / 成本函數

def object_function(y_prediction , y_true) :

    mse = numpy.mean((y_true - y_prediction)**2)

    return mse


# 3. 設定起始和終止條件

# a. 假設參數 a , b 一開始皆為 0

a = 0
b = 0

# b. 假設試行 10000 次 , 學習率為 0.001 , mse 為無限大

iteration = 10000
learning_rate = 0.001
previous_mse = float("inf")

# c. 假設終止條件是目標函數差別 < 0.000001

mse_threhold = 0.000001


# 4. 開始迴圈進行逼近

for i in range(iteration) : 

    y_hat = a * x + b

    da = -2 * numpy.mean(x*(y-y_hat))

    db = -2 * numpy.mean((y-y_hat))

    a = a - learning_rate*da

    b = b - learning_rate*db

    if (i+1)%1000 == 0 : 
        
        print(f"試行第{i+1}次，a為{a}，b為{b},mse為{previous_mse}")

    y_hat = a * x + b

    new_mse = object_function(y_hat,y)

    if previous_mse - new_mse < mse_threhold : 

        break

    previous_mse = new_mse

print(f"試行次數為{i+1}次，最後結果為 a = {a}，b = {b}，mse = {new_mse}，模型為 y = {a}*x + {b}")


試行第1000次，a為2.1096675206838262，b為0.8097539923224204,mse為0.22254104580342302
試行第2000次，a為2.067206887395702，b為1.0980359534544855,mse為0.1356049347531842
試行第3000次，a為2.0421456095275787，b為1.2681868384701027,mse為0.10531950798815264
試行第4000次，a為2.0273538459634，b為1.3686139465621519,mse為0.09476915022243615
試行第5000次，a為2.018623394527926，b為1.4278884195618446,mse為0.09109378352385189
試行次數為5691次，最後結果為 a = 2.014783760598431，b = 1.4539572064240953，mse = 0.09007607919129233，模型為 y = 2.014783760598431*x + 1.4539572064240953


In [78]:
# 做一個直接用 scikit learn 套件算迴歸參數的版本


import numpy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression


# 1. 產生 ( x , y ) 資料

# a. 直接自己做

# 要記得把 x , y 轉成 2D 陣列，即便只有一排資料，這樣 regression 才能跑

# x 要做 reshape(-1,1) , 第一個參數代表選取所有列，第二個是欄 , y 也是一樣 , 但 random 要用 tuple ,

# 隨機變數產生可以選 random.random (只會產生 0 到 1 數字) 或 random.uniform (可以自己選上下數字範圍)

x = numpy.linspace(0,10,20).reshape(-1,1)

y = 2*x + 1 + numpy.random.uniform(-1,1,20).reshape(-1,1)

# b. 用模組的方法做

# x , y = make_regression(n_samples=20,n_features=1,noise=10,random_state=42)


# 2. 做訓練資料集和測試資料集

x_test , x_train , y_test , y_train = train_test_split(x,y,test_size=0.2,random_state=42)


# 3. 進行迴歸

model = LinearRegression()

model.fit(x,y)


# 4. 產出結果

print(f"a參數 = {model.coef_} , b參數 = {model.intercept_} , 模型 R square 為 {model.score(x_test,y_test)}")



a參數 = [[1.89057145]] , b參數 = [1.50187258] , 模型 R square 為 0.9937832526172294


In [87]:
# 改用時間序列資料以及 statsmodels 來做 OLS 迴歸，並產生 p-value 和 F 指標來判斷模型好壞


import numpy
from statsmodels.api import add_constant , OLS
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


# 1. 先載入股價資料

stock_price = numpy.array([6449.80,6468.54,6466.58,6445.76,6373.45,6389.45,6340.00,6345.06,6299.19,6329.94,6238.01,6339.39,6362.90,6370.86,6389.77,6388.64,6363.35,6358.91,6309.62,6305.60,6296.79,6297.36,6263.70,6243.76])


# 2. 做出時間序列的訓練和測試資料集，為今天和前幾天的資料作為解釋變數

# a.先區分訓練集和測試集

# 算 array 的元素總數可以先做成切片再來計算，不然只能算一維的

N = len(stock_price)
stock_price_train = stock_price[:int(numpy.floor(N*0.6)),]
stock_price_test = stock_price[int(numpy.floor(N*0.6)):,]

# b. 再做成變數資料

y_train = []
X_1_train = []
X_2_train = []

N_train = len(stock_price_train)

for i in range(N_train-2) : 

    y_train.append(stock_price_train[i])
    X_1_train.append(stock_price_train[i+1])
    X_2_train.append(stock_price_train[i+2])

y_test = []
X_1_test = []
X_2_test = []

N_test = len(stock_price_test)

for i in range(N_test-2) : 

    y_test.append(stock_price_train[i])
    X_1_test.append(stock_price_train[i+1])
    X_2_test.append(stock_price_train[i+2])


# 3. 把資料合併並加上常數項

# stack 的資料來源可以是 list，做完就轉成 numpy array

# axis = 0 是往 x 軸展開，基本上就是 append，也可以看做一列一列增加原陣列

# axis = 1 or -1 是往 y 軸展開，這個比較特別，會變成欄和列互換，就是 x 軸展開後再做矩陣反轉，陣列的大小會改變

x_train_data = numpy.stack((X_1_train,X_2_train),axis=-1)

x_train_set = add_constant(x_train_data)

x_test_data = numpy.stack((X_1_test,X_2_test),axis=-1)


# 4. 做 OLS 回歸 , 結果和 scikit learn 一樣

model = OLS(y_train,x_train_set)
result = model.fit()

model_2 = LinearRegression()
model_2.fit(x_train_data,y_train)

print(result.summary())
print(model_2.coef_,model_2.intercept_)


# 5. 做樣本外 R square 測試 , OLS 只能用 scikit learn 的 r2_score 來算配適程度

x_test_set = add_constant(x_test_data)

y_prediction = model.predict(x_test_data)

print(y_prediction)

print(r2_score(y_test))

print(model_2.score(x_test_data,y_test))






                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.476
Method:                 Least Squares   F-statistic:                     5.988
Date:                Sun, 17 Aug 2025   Prob (F-statistic):             0.0222
Time:                        08:38:06   Log-Likelihood:                -62.836
No. Observations:                  12   AIC:                             131.7
Df Residuals:                       9   BIC:                             133.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        591.2658   1726.740      0.342      0.7

c:\Users\zxzxa\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=12 observations were given.
  return hypotest_fun_in(*args, **kwds)


ValueError: shapes (12,3) and (8,2) not aligned: 3 (dim 1) != 8 (dim 0)